In [1]:
import json
import pandas as pd
from sentence_transformers import (SentenceTransformer, util)

In [2]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [14]:
with open('Data/exiobase_sectors.json','r') as f:
    exio_sectors = json.load(f)
exio_embeddings = model.encode([i[1] for i in exio_sectors])

In [15]:
products = ['ADPE System Configuration','Chocolate','Soundcloud subscription','Geophysical Instruments']
products_embeddings = model.encode(products)

In [16]:
scores = util.pytorch_cos_sim(products_embeddings, exio_embeddings)
sorted_scores, indices = scores.sort(dim=1, descending=True)

In [28]:
df_results = pd.DataFrame(None, ['order', 'sector', 'similarity'])

number_of_matches = 10

for i, product in enumerate(products):
    for j in range(0, number_of_matches):
        df_results = pd.concat([df_results, 
                   pd.DataFrame([product, 
                                 j+1,
                                 exio_sectors[indices[i][j].cpu().numpy()][0],
                                 #exio_embeddings[indices[i][j].cpu().numpy()],
                                 exio_sectors[indices[i][j].cpu().numpy()][1],
                                 sorted_scores[i][j].cpu().numpy().tolist()],
                                ['product', 'order', 'code sector', 'sector', 'similarity'])],
                               axis=1)
        
df_results = df_results.T.set_index(['product', 'order'])

In [29]:
df_results

sector  \
product                   order                                                      
ADPE System Configuration 1      Planning, creating and placement services of a...   
                          2                        Systems and user tools software   
                          3           Installation services of musical instruments   
                          4              Installation services of office machinery   
                          5              Installation services of office machinery   
                          6                  Assembly services of computer modules   
                          7                  Assembly services of computer modules   
                          8                              Joinery installation work   
                          9                              Joinery installation work   
                          10     Administrative multipurpose development projec...   
Chocolate                 1                      Chocolate and sugar confectionery   
                          2               Cocoa; chocolate and sugar confectionery   
                          3                                                  Cocoa   
                          4      Chocolate and food preparations containing coc...   
                          5      Chocolate and food preparations containing coc...   
                          6                                                  Sugar   
                          7                                            Cocoa beans   
                          8                                                   Whey   
                          9                                                Cereals   
                          10     Sugar confectionery (including white chocolate...   
Soundcloud subscription   1               Reproduction services of sound recording   
                          2               Reproduction services of sound recording   
                          3               Reproduction services of sound recording   
                          4                                       Sound recordings   
                          5      Leasing or rental services of musical instruments   
                          6                                         Radio services   
                          7               Industrial services for sound recordings   
                          8               Industrial services for sound recordings   
                          9      Records, tapes, CDs, DVDs and other recorded m...   
                          10                                Printed music; printed   
Geophysical Instruments   1      Navigational, meteorological, geophysical and ...   
                          2      Surveying, hydrographic, oceanographic, hydrol...   
                          3      Instrument for checking other physical charact...   
                          4      Instruments and apparatus for measuring or che...   
                          5      Instruments for measuring electrical quantitie...   
                          6      Instruments and appliances for measuring, chec...   
                          7      Instruments and appliances for measuring, chec...   
                          8      Instruments and apparatus for physical or chem...   
                          9      Parts of instruments and appliances for measur...   
                          10     Geological, geophysical and other scientific p...   

                                similarity code sector  
product                   order                         
ADPE System Configuration 1       0.301748    74.40.12  
                          2       0.296192    72.21.11  
                          3       0.293924    36.30.91  
                          4       0.282947     30.01.9  
                          5       0.282947    30.01.90  
                          6        0.27624     30.02.2  
       